In [ ]:
import datetime
import os
from pathlib import Path

import pandas
import requests
import tqdm
from bs4 import BeautifulSoup
from pandas import DataFrame

In [ ]:
ROOT_DIR = Path(os.getcwd()).parent.absolute().__str__()
ROOT_DIR

In [ ]:
road_dataframe = pandas.read_csv(f'{ROOT_DIR}/data/raw/road_info.csv')
road_dataframe

In [ ]:
road_list = road_dataframe['chinese_district_name']
road_list

In [ ]:
url = 'https://programme.rthk.hk/channel/radio/trafficnews/index.php?d={date}'

start_date = datetime.datetime(2010, 1, 1)

info = {
    'date': [],
    'time': [],
    'location': [],
    'detail': [],
}

for i in tqdm.tqdm(range(0, 4018)):
    news_date = (start_date + datetime.timedelta(i)).strftime('%Y%m%d')
    response = requests.get(url.format(date=news_date))
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find('div', {'class': 'articles'})
    ul = articles.find_all('li', {'class': 'inner'})
    if ul:
        for li in ul:
            data = li.get_text().split('\t')
            detail = data[0]
            date = data[-1].strip().split(' HKT ')[0]
            time = data[-1].strip().split(' HKT ')[1]
            for road in road_list:
                if road in detail:
                    info['date'].append(str(date))
                    info['time'].append(str(time))
                    info['location'].append(str(road))
                    info['detail'].append(str(detail))

In [ ]:
news_dataframe = DataFrame(info)
news_dataframe

In [ ]:
news_dataframe.to_csv(f'{ROOT_DIR}/data/raw/news_info.csv', index=False)